In [2]:
import pandas as pd

In [3]:
# Read json files into dataframe
purch_data = pd.read_json('purchase_data.json')
purch_data2 = pd.read_json('purchase_data2.json')

In [4]:
# Combine data frames into one
frames = [purch_data, purch_data2]
master_df = pd.concat(frames)
master_df.head(10)

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59
5,20,Male,10,Sleepwalker,1.73,Tanimnya91
6,20,Male,153,Mercenary Sabre,4.57,Undjaskla97
7,29,Female,169,"Interrogator, Blood Blade of the Queen",3.32,Iathenudil29
8,25,Male,118,"Ghost Reaver, Longsword of Magic",2.77,Sondenasta63
9,31,Male,99,"Expiration, Warscythe Of Lost Worlds",4.53,Hilaerin92


In [6]:
# Drop any NA rows and get player count
master_df.dropna(how='any')
player_count = len(master_df['SN'].unique())
pc_df = pd.DataFrame({'Total Players': [player_count]})
pc_df

,Total Players
0,612


In [129]:
# Purchasing Analysis
unique_item_count = len(master_df['Item Name'].unique())
avg_purch_price = round(master_df['Price'].mean(),2)
purchases = len(master_df.index)
total_revenue = round(master_df['Price'].sum(),2)
# Create output dataframe
output1 = pd.DataFrame({
    'Number of Unique Items': [unique_item_count], 
    'Average Price': '$'+str(avg_purch_price), 
    'Number of Purchases': purchases, 
    'Total Revenue': '$'+str(total_revenue)
})
output1 = output1[['Number of Unique Items', 'Average Price', 'Number of Purchases', 'Total Revenue']]
output1

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,180,$2.93,858,$2514.43


In [8]:
# Gender Demographics
male_count = master_df['Gender'].value_counts()['Male']
female_count = master_df['Gender'].value_counts()['Female']
other_count = master_df['Gender'].value_counts()['Other / Non-Disclosed']
male_percent = round(male_count / purchases*100, 2)
female_percent = round(female_count / purchases*100, 2)
other_percent = round(other_count / purchases*100, 2)
output2 = pd.DataFrame(
    data=[[female_percent, female_count], [male_percent, male_count], [other_percent, other_count]], 
    columns=['Percentage of Players', 'Total Count'], 
    index=['Female', 'Male', 'Other / Non-Disclosed']
)
output2

,Percentage of Players,Total Count
Female,17.37,149
Male,81.24,697
Other / Non-Disclosed,1.40,12


In [9]:
# Purchasing Analysis (Gender)
group_count = master_df.groupby('Gender').count()
output3 = group_count[['Age']]
output3.columns = ['Purchase Count']
output3['Average Purchase Price'] = master_df.groupby('Gender').mean()[['Price']]
output3['Total Purchase Value'] = master_df.groupby('Gender').sum()[['Price']]
output3['Average Purchase Price'] = output3['Average Purchase Price'].apply(lambda x: '$'+str(round(x,2)))
output3['Total Purchase Value'] = output3['Total Purchase Value'].apply(lambda x: '$'+str(round(x,2)))
output3




C:\Users\jakec\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\jakec\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\jakec\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

,Purchase Count,Average Purchase Price,Total Purchase Value
Gender,,,
Female,149,$2.85,$424.29
Male,697,$2.94,$2052.28
Other / Non-Disclosed,12,$3.15,$37.86


In [10]:
master_df['Age'].max()

45

In [62]:
# Age demographics
bins = [0, 10, 15, 20, 25, 30, 35, 40, 150]

bin_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', 
             '40+']

master_df['Age Group'] = pd.cut(master_df['Age'], bins, right=False, labels=bin_names)

output4 = master_df.groupby('Age Group').count()[['Age']]
output4.columns = ['Purchase Count']
output4['Average Purchase Price'] = master_df.groupby('Age Group').mean()['Price']
output4['Total Purchase Value'] = master_df.groupby('Age Group').sum()['Price']
output4['Average Purchase Price'] = output4['Average Purchase Price'].apply(lambda x: '$'+str(round(x,2)))
output4['Total Purchase Value'] = output4['Total Purchase Value'].apply(lambda x: '$'+str(round(x,2)))
output4


,Purchase Count,Average Purchase Price,Total Purchase Value
Age Group,,,
<10,33,$2.95,$97.28
10-14,38,$2.79,$105.91
15-19,144,$2.89,$416.83
20-24,372,$2.92,$1087.66
25-29,134,$2.96,$396.44
30-34,71,$2.97,$211.14
35-39,48,$2.93,$140.77
40+,18,$3.24,$58.4


In [61]:
# Top Spenders
top_spenders = master_df.groupby('SN').sum()[['Price']]
top_spenders = top_spenders.sort_values('Price', ascending=False)
top_spenders_mean = master_df.groupby('SN').mean()[['Price']]
top_spenders_mean
output5 = pd.merge(top_spenders, top_spenders_mean, how='left', left_index=True, right_index=True)
top_spenders_count = master_df.groupby('SN').count()[['Price']]
output5 = pd.merge(output5, top_spenders_count, how='left', left_index=True, right_index=True)
output5.columns = ['Total Purchase Value', 'Average Purchase Price', 'Purchase Count']
output5 = output5[['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']]
output5 = output5.iloc[:5]
output5['Average Purchase Price'] = output5['Average Purchase Price'].apply(lambda x: '$'+str(round(x,2)))
output5['Total Purchase Value'] = output5['Total Purchase Value'].apply(lambda x: '$'+str(round(x,2)))
output5

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Aerithllora36,4,$3.77,$15.1
Saedue76,4,$3.39,$13.56
Sondim43,4,$3.25,$13.02
Mindimnya67,4,$3.18,$12.74


In [112]:
# Most Popular Items
top_items = master_df.groupby(['Item Name', 'Item ID']).count()[['Price']].sort_values('Price', ascending=False)
top_items.reset_index(inplace=True)
top_items.columns = ['Item Name', 'Item ID', 'Purchase Count']
master_df1 = master_df[['Item Name', 'Price']]
output6 = top_items.merge(master_df1, on='Item Name', how='left')
output6.drop_duplicates(subset='Item Name', inplace=True)
top_items2 = master_df1.groupby('Item Name').sum()
top_items2.reset_index(inplace=True)
top_items2.columns = ['Item Name', 'Total Purchase Value']
output6 = output6.merge(top_items2, on='Item Name', how='left')
output6 = output6[['Item ID', 'Item Name', 'Purchase Count', 'Price', 'Total Purchase Value']]
output6 = output6.iloc[:5]
output6['Price'] = output6['Price'].apply(lambda x: '$'+str(round(x,2)))
output6['Total Purchase Value'] = output6['Total Purchase Value'].apply(lambda x: '$'+str(round(x,2)))
output6


,Item ID,Item Name,Purchase Count,Price,Total Purchase Value
0,84,Arcane Gem,12,$2.23,$29.34
1,39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
2,31,Trickster,10,$2.07,$23.22
3,154,Feral Katana,9,$2.19,$23.55
4,13,Serenity,9,$1.49,$13.41


In [127]:
# Most Profitable Items
prof_items = master_df.groupby(['Item Name', 'Item ID']).sum()[['Price']].sort_values('Price', ascending=False)
prof_items.reset_index(inplace=True)
prof_items.columns = [['Item Name', 'Item ID', 'Total Purchase Value']]
group_count = master_df.groupby('Item Name').count()[['Price']]
group_count.reset_index(inplace=True)
group_count.columns = ['Item Name', 'Purchase Count']
output7 = prof_items.merge(group_count, how='left', on='Item Name')
output7 = output7.merge(master_df1, how='left', on='Item Name')
output7.drop_duplicates(subset='Item Name', inplace=True)
output7 = output7[['Item ID', 'Item Name', 'Purchase Count', 'Price', 'Total Purchase Value']]
output7 = output7.iloc[:5]
output7['Price'] = output7['Price'].apply(lambda x: '$'+str(round(x,2)))
output7['Total Purchase Value'] = output7['Total Purchase Value'].apply(lambda x: '$'+str(round(x,2)))
output7

,Item ID,Item Name,Purchase Count,Price,Total Purchase Value
0,34,Retribution Axe,9,$4.14,$37.26
9,107,"Splitter, Foe Of Subtlety",9,$3.61,$33.03
18,115,Spectral Diamond Doomblade,7,$4.25,$29.75
25,32,Orenmir,6,$4.95,$29.7
31,84,Arcane Gem,12,$2.23,$29.34
